In [73]:
# imports
import typing
import os
import math
import tensorflow as tf
from tensorflow import keras
import numpy as np
from numpy import genfromtxt
from numpy.lib import recfunctions as rfn
import pandas as pd
import sklearn
from sklearn.preprocessing import MinMaxScaler
#print(tf.__version__)

In [74]:
# colab config
#from google.colab import drive
#drive.mount('/content/drive')
#os.chdir('/content/drive/My Drive/cs470')

# Data Preprocessing

## Data Loading

The dataset was downloaded from Kaggle. It contains 32 Columns of 1,000,000 entries. Since it is 216.59 MB, it may take time to load.

_in this project, "label" will refer to column categories, and "data_actual" will refer to the actual value of the data_

In [75]:
data = pd.read_csv("Base.csv") # load csv from drive
classification = data.pop("fraud_bool").tolist() # remove classification label from dataset
labels = data.keys().tolist() # get column labels

# One-hot vectorization

Machine learning algorithms have a hard time understanding strings. To confront this, we replace a column of multiple unique string values with multiple columns for each unique category. These columns contain a boolean to indicate which category it was. This is alternatively known as a _One-hot_.

In [76]:
# one-hotify labels
labels_categorical = ["payment_type", "employment_status", "housing_status", "source", "device_os"] # list that contains columns to be binarized
labels_vectorized = [] # list that contains all newly created binarized columns
for label in labels_categorical:
    label_index = data.columns.get_loc(label)
    column_binarized = pd.get_dummies(data[label])
    for label_binarized in column_binarized:
        # new label joins the category with the original column name
        label_binarized_new = label + "_" + label_binarized
        data.insert(
            label_index,
            label_binarized_new,
            column_binarized[label_binarized])
        labels_vectorized.append(label_binarized_new)
    del data[label]
# prove that removal occurred and that new columns were added
assert("payment_type" not in data.columns and "device_os" not in data.columns and "device_os_windows" in data.columns)

# Minimum-Maximum Normalization

The backpropagation algorithm will eventually attempt to normalize the range of values within a column to become from 0 to 1. We can save time by preprocessing the data beforehand, shaving seconds if not minutes off of training.

In [77]:
# minimum-maximum normalization
scaler = MinMaxScaler() # scales data between 0 and 1
for label in labels:
    if label not in labels_categorical: # check only the ones that were not categorical (this implies columns of numbers, not strings)
        data[label] = scaler.fit_transform(data[[label]])

# Data Partitioning

Finally, the data will be partitioned into both training and evaluation subsets.

In [78]:
# data partitioning
training_ratio = 0.85 # this percent will be used for training
training_index = math.floor(len(data)*training_ratio)
data_training = (data[:training_index]) # training data
labels_training = (classification[:training_index]) # training data's actual value
data_evaluation = (data[training_index:]) # evaluation data
labels_evaluation  = (classification[training_index:]) # evaluation data's actual value

# Model Design

We will be using a simple Multilayer Neural Network for this project. This is because our data is simply just numbers, which have no spatial significance (like an image), so we will not be using a Convolutional Neural Network.

In [79]:
# model design
model: keras.Model = keras.Sequential([
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')]) # output layer

# Model Compilation

In [80]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=[tf.keras.metrics.BinaryAccuracy(),tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), tf.keras.metrics.FalsePositives(), tf.keras.metrics.FalseNegatives(), tf.keras.metrics.TrueNegatives(), tf.keras.metrics.TruePositives()])

# Model Training

In [81]:
history = model.fit(
    x=data_training,
    y=labels_training,
    epochs=3)

ValueError: Failed to find data adapter that can handle input: <class 'pandas.core.frame.DataFrame'>, (<class 'list'> containing values of types {"<class 'int'>"})

In [ ]:
history = model.evaluate(evaluation_dataset, evaluation_dataset_labels)

4688/4688 [==============================] - 28s 6ms/step - loss: 0.0413 - binary_accuracy: 0.9903 - precision: 0.5000 - false_positives: 3.0000


In [ ]:
model.predict(np.array([evaluation_dataset[5]]))

2022-12-11 16:07:14.649445: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 290ms/step


array([[0.00150064]], dtype=float32)